In [1]:
!pip3 install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 780.8 kB/s eta 0:00:00m eta 0:00:010:01:02

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import numpy as np
from wrapper import compute_check_to_value  # tu wrapper Cython

def show_matrix(L, pcm_matrix, rows, cols):
    print("Initial L Matrix:")
    for i in range(rows):
        row_str = []
        for j in range(cols):
            if pcm_matrix[i][j] == 1:
                row_str.append(f"{L[i][j]:.2f}")
            else:
                row_str.append("0.00")
        print(" ".join(row_str))


def main():
    with open("input3.txt", "r") as f:
        # Leer p
        p = float(f.readline().strip())
        p = 1.0 - (2.0/3.0) * p

        # Leer filas y columnas
        rows, cols = map(int, f.readline().split())

        # Inicializar matrices
        pcm_matrix = np.zeros((rows, cols), dtype=int)
        L = np.zeros((rows, cols), dtype=float)

        # Leer pcm_matrix y L
        for i in range(rows):
            values = list(map(int, f.readline().split()))
            for j, value in enumerate(values):
                pcm_matrix[i, j] = value
                if value == 1:
                    L[i, j] = p
                else:
                    L[i, j] = 0.0

        # Leer syndrome
        syndrome = list(map(int, f.readline().split()))
        print("Initial Syndrome:", " ".join(map(str, syndrome)))

        # Inicializar Lj
        Lj = np.full(cols, p, dtype=float)
        print(f"Lj: {p:.2f}")

        # Leer alpha
        alpha = float(f.readline().strip())
        print(f"Alpha: {alpha:.2f}")

        # Leer número de iteraciones
        num_it = int(f.readline().strip())
        print(f"Max Iterations: {num_it}\n")

        # Mostrar L inicial
        show_matrix(L, pcm_matrix, rows, cols)

        # Llamar a la función de Cython
        compute_check_to_value(L, pcm_matrix, syndrome, rows, cols, Lj, alpha, num_it)


if __name__ == "__main__":
    main()
